# Comparing Nonlinear Projection Algorithms
One of the first things I wanted to do after developing a hypothesis testing analytic with RBO was to
compare the performance of my favorite local structure preserving nonlinear projection algorithm [UMAP](https://en.wikipedia.org/wiki/Nonlinear_dimensionality_reduction#Uniform_manifold_approximation_and_projection) and an
alternative algorithm [PACMAP](https://github.com/YingfanWang/PaCMAP).

The idea is similar to how one can compare document embedder maps:

* Take a high dimensional data set $D\subset \mathbb{R}^P$ with $P \gg 1$.
* Select a target dimension $Q \ll P$ to project down to.
* Project the data down to $X = UMAP(D) \subset \mathbb{R}^Q$ and $Y = PACMAP(D) \subset \mathbb{R}^Q$
* Just as in the document embedder scenario randomly select $M$ data points $\{D_m\}$ and look at the corresponding neighborhoods for $\{X_m\}$ anb $\{Y_m\}$.
* For comparing local structure preserving algorithms,
  * use the Euclidean metric to find and rank sort the $K$ nearest neighbors to each poin $X_m$ (respectively $Y_m$).
  * To determine an appropriate $K$ for each $X_m$ you can use the distribution of sorted *ascending* distances from each projected point to $X_m$ (exclude $X_m$ itself) and find the elbow in the plot. The length of the left segment is the value of $K$ to use. 
  * For each ranked list of $K$ points use the geometric probability distribution as in the document embedder scenario.
  * Now that you have lists of lists for both maps and the corresponding probability distribution for each list, you can use the compute_recommender_test_statistic function to perform hypothesis testing.
```python
import rbo_analytics
Z = rbo_analytics.compute_recommender_test_statistic(lists_a, lists_b,probs,verbose=True)

print("Sigmage that the 2 nonlinear projection algorithms are functionally related when it comes to preserving local structure: {Z}")
print(f"The 2 nonlinear projection algorithms are functionally related when it comes to preserving local structure: {Z>=-2.33}")
```

* For comparing their global structure preserving behavior, we invert the Euclidean metric: $||a - b|| \rightarrow 1/||a-b||$ to reoder the rankings and proceed as in the local structure preserving analysis.


In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import rbo_analytics
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

In [ ]:
from umap import UMAP
from pacmap import PaCMAP

## Data set
I'm using the Scikit-Learn Handwritten digits data set for comparing the 2 nonlinear projection systems.

In [ ]:
digits = sklearn.datasets.load_digits()
data = digits['data']

print(f"# of data points: {data.shape[0]}")
print(f"Extrinsic dimension of the data set: {data.shape[1]}")

## Constructing projection maps

In [ ]:
umap_model = UMAP(n_components=3)
pacmap_model = PaCMAP(n_components=3)

In [ ]:
%%time 
umap_projected = umap_model.fit_transform(data)

In [ ]:
%%time
pacmap_projected = pacmap_model.fit_transform(data)

## Constructing the lists
I'll randomly selected 100 data points and compare the local and global neighborhoods under projection from
both projection models.

In [ ]:
X = (umap_projected**2).sum(axis=1).reshape(-1,1)
distances_umap= X + X.T - 2*umap_projected.dot(umap_projected.T)
distances_umap[distances_umap<0.0] = 0.0
distances_umap =distances_umap**0.5

In [ ]:
Y = (pacmap_projected**2).sum(axis=1).reshape(-1,1)
distances_pacmap= Y + Y.T - 2*pacmap_projected.dot(pacmap_projected.T)
distances_pacmap[distances_pacmap<0.0] = 0.0
distances_pacmap =distances_pacmap**0.5

In [ ]:
N = data.shape[0]
ks = np.random.choice(N,size=100,replace=False)
sampled_distance_rows = distances_umap[ks]

In [ ]:
sorted_rows = sampled_distance_rows.copy()
sorted_rows.sort(axis=1)

A quick plot of distances to neighboring pointd for a couple of selected projected data points.

In [ ]:
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(sorted_rows[0]);

plt.subplot(1,2,2)
plt.plot(sorted_rows[5]);

I'm going to select my neighborhood structures to be determined by the 30 nearest points.

In [ ]:
Ks = np.ones(100,dtype=np.int32)*30

# Comparing Local Structure Preservation

In [ ]:
def generate_geometric_probs(K):
    """Generates a probability distribution, Pr(x = n) ~ p**n """

    p = (0.01)**(1/K)
    probs = p**np.arange(K)
    probs = probs/probs.sum()
    return probs

I'm going to generate the list of local points under both projection maps and a geometric probability distribution for UMAP's selection of points.

In [ ]:
lists_a = [((distances_umap[n]).argsort()[1:Ks[t]]).tolist() for t,n in enumerate(ks)]
lists_b = [((distances_pacmap[n]).argsort()[1:Ks[t]]).tolist() for t,n in enumerate(ks)]

In [ ]:
probs_a  = [generate_geometric_probs(K-1) for K in Ks]

Here is where I compute the $Z$ statistic and decide if the 2 algorithms preserve local structure in a related fashion:

In [ ]:
Z = rbo_analytics.compute_recommender_test_statistic(lists_a, lists_b,probs_a,verbose=True)

print(f"Sigmage that the 2 nonlinear projection algorithms are functionally related when it comes to preserving local structure: {Z}")

if Z >= -2.33:
    print("The 2 nonlinear projections algorithms are functionally related when it comes to preserving global structure.")
else:
    print("The 2 nonlinear projections algorithms are *NOT* functionally related when it comes to preserving global structure.")


# Comparing Global Structure Preservation

In [ ]:
lists_a = [((-distances_umap[n]).argsort()[0:Ks[t]]).tolist() for t,n in enumerate(ks)]
lists_b = [((-distances_pacmap[n]).argsort()[0:Ks[t]]).tolist() for t,n in enumerate(ks)]

In [ ]:
probs_a  = [generate_geometric_probs(K) for K in Ks]

Here is where I compute the $Z$ statistic and decide if the 2 algorithms preserve global structure in a related fashion:

In [ ]:
Z = rbo_analytics.compute_recommender_test_statistic(lists_a, lists_b,probs_a,verbose=True)

print(f"Sigmage that the 2 nonlinear projection algorithms are functionally related when it comes to preserving local structure: {Z}")

if Z >= -2.33:
    print("The 2 nonlinear projections algorithms are functionally related when it comes to preserving global structure.")
else:
    print("The 2 nonlinear projections algorithms are *NOT* functionally related when it comes to preserving global structure.")
